# Создание таблицы и получение выборки данных
## Содержание
В этом уроке будет рассмотрен синтаксис и семантика следующих SQL запросов:

* создание таблиц и вставка значений;
* выборка всех данных из таблицы;
* выборка данных из отдельных столбцов;
* выборка отдельных столбцов и присвоение им новых имен;
* создание вычисляемых столбцов;
* вычисляемые столбцы, математические функции;
* вычисляемые столбцы, логические функции;
* выборка данных по простому условию;
* выборка данных с использованием логических выражений и операций;
* выборка данных, операторы BETWEEN, IN;
* выборка текстовых данных по шаблону, оператор LIKE;
* выборка данных с сортировкой.

Структура и наполнение таблицы

Все запросы будут формулироваться для таблицы book:

![table](./images/sql_intro_book_table.png)

## Создание таблицы и заполнение

In [9]:
import pandas as pd
import numpy as np
import mysql.connector

In [10]:
config = {
        'user': 'root',
        'password': 'root',
        'host': 'db',
        'port': 3306,
        'database': 'SQLIntro',
        'charset': 'utf8'
    }

In [11]:
def create_table(query: str) -> None:
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close()
    connection.close()

In [12]:
def insert_values(query: str) -> None:
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()
    
    try:
        cursor.execute(query)
        connection.commit()
    except Exception as e:
        print(repr(e))
        connection.rollback()
    finally:
        cursor.close()
        connection.close()

In [13]:
# query = '''
#         CREATE TABLE book(
#                             book_id INT PRIMARY KEY AUTO_INCREMENT,
#                             title   VARCHAR(50),
#                             author  VARCHAR(30),
#                             price   DECIMAL(8, 2),
#                             amount  INT
#                          );
#         '''
# create_table(query)

In [14]:
# query = '''
#         INSERT INTO book (title, author, price, amount) VALUES ('Мастер и Маргарита', 'Булгаков М.А.', 670.99, 3),
#                                                                ('Белая гвардия', 'Булгаков М.А.', 540.50, 5),
#                                                                ('Идиот', 'Достоевский Ф.М.', 460.00, 10),
#                                                                ('Братья Карамазовы', 'Достоевский Ф.М.', 799.01, 2),
#                                                                ('Игрок', 'Достоевский Ф.М.', 480.50, 10),
#                                                                ('Стихотворения и поэмы', 'Есенин С.А.', 650.00, 15);

#         '''
# insert_values(query)

## Выборка данных

In [15]:
def select_data(query: str) -> pd.DataFrame:
    connection = mysql.connector.connect(**config)
    df_mysql = pd.read_sql(query, con=connection)
    connection.close()
    return df_mysql

In [16]:
query = '''
        SELECT *
        FROM book;
        '''

select_data(query).style.hide_index()

book_id,title,author,price,amount,buy
1,Мастер и Маргарита,Булгаков М.А.,670.990000,3,0
2,Белая гвардия,Булгаков М.А.,513.480000,12,3
3,Идиот,Достоевский Ф.М.,387.400000,13,8
4,Братья Карамазовы,Достоевский Ф.М.,799.010000,2,0
5,Игрок,Достоевский Ф.М.,432.450000,10,0
6,Стихотворения и поэмы,Есенин С.А.,650.000000,15,15
7,Лирика,Пастернак Б.Л.,518.990000,4,0
8,Черный человек,Есенин С.А.,541.690000,12,0


In [17]:
query = '''
        SELECT author, title, price
        FROM book;
        '''

select_data(query).style.hide_index()

author,title,price
Булгаков М.А.,Мастер и Маргарита,670.990000
Булгаков М.А.,Белая гвардия,513.480000
Достоевский Ф.М.,Идиот,387.400000
Достоевский Ф.М.,Братья Карамазовы,799.010000
Достоевский Ф.М.,Игрок,432.450000
Есенин С.А.,Стихотворения и поэмы,650.000000
Пастернак Б.Л.,Лирика,518.990000
Есенин С.А.,Черный человек,541.690000


In [18]:
query = '''
        SELECT title as Название, author as Автор
        FROM book;
        '''

select_data(query).style.hide_index()

Название,Автор
Мастер и Маргарита,Булгаков М.А.
Белая гвардия,Булгаков М.А.
Идиот,Достоевский Ф.М.
Братья Карамазовы,Достоевский Ф.М.
Игрок,Достоевский Ф.М.
Стихотворения и поэмы,Есенин С.А.
Лирика,Пастернак Б.Л.
Черный человек,Есенин С.А.


In [19]:
query = '''
        SELECT title, amount, amount*1.65 AS pack
        FROM book;
        '''
select_data(query).style.hide_index()

title,amount,pack
Мастер и Маргарита,3,4.950000
Белая гвардия,12,19.800000
Идиот,13,21.450000
Братья Карамазовы,2,3.300000
Игрок,10,16.500000
Стихотворения и поэмы,15,24.750000
Лирика,4,6.600000
Черный человек,12,19.800000


In [20]:
query = '''
        SELECT title, author, amount, round(price*0.7, 2) AS new_price
        FROM book;
        '''

select_data(query).style.hide_index()

title,author,amount,new_price
Мастер и Маргарита,Булгаков М.А.,3,469.690000
Белая гвардия,Булгаков М.А.,12,359.440000
Идиот,Достоевский Ф.М.,13,271.180000
Братья Карамазовы,Достоевский Ф.М.,2,559.310000
Игрок,Достоевский Ф.М.,10,302.720000
Стихотворения и поэмы,Есенин С.А.,15,455.000000
Лирика,Пастернак Б.Л.,4,363.290000
Черный человек,Есенин С.А.,12,379.180000


In [21]:
query = '''
        SELECT author, title,
               round(IF (author='Булгаков М.А.', price*1.10, IF(author='Есенин С.А.', price*1.05, price)), 2) AS new_price
        FROM book;
        '''

select_data(query).style.hide_index()

author,title,new_price
Булгаков М.А.,Мастер и Маргарита,738.090000
Булгаков М.А.,Белая гвардия,564.830000
Достоевский Ф.М.,Идиот,387.400000
Достоевский Ф.М.,Братья Карамазовы,799.010000
Достоевский Ф.М.,Игрок,432.450000
Есенин С.А.,Стихотворения и поэмы,682.500000
Пастернак Б.Л.,Лирика,518.990000
Есенин С.А.,Черный человек,568.770000


In [22]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE amount < 10;
        '''

select_data(query).style.hide_index()

author,title,price
Булгаков М.А.,Мастер и Маргарита,670.990000
Достоевский Ф.М.,Братья Карамазовы,799.010000
Пастернак Б.Л.,Лирика,518.990000


In [23]:
query = '''
        SELECT title, author, price, amount
        FROM book
        WHERE price*amount >= 5000 and (price < 500 or price > 600);
        '''

select_data(query).style.hide_index()

title,author,price,amount
Идиот,Достоевский Ф.М.,387.400000,13
Стихотворения и поэмы,Есенин С.А.,650.000000,15


In [24]:
query = '''
        SELECT title, author
        FROM book
        WHERE (price BETWEEN 540.50 and 800) and    amount IN (2, 3, 5, 7);
        '''

select_data(query).style.hide_index()

title,author
Мастер и Маргарита,Булгаков М.А.
Братья Карамазовы,Достоевский Ф.М.


In [25]:
query = '''
        SELECT title, author
        FROM book
        WHERE title LIKE '% %' AND author LIKE '%С.%';
        '''

select_data(query).style.hide_index()

title,author
Стихотворения и поэмы,Есенин С.А.
Черный человек,Есенин С.А.


In [26]:
query = '''
        SELECT author, title
        FROM book
        WHERE amount BETWEEN 2 AND 14 
        ORDER BY author DESC, title ASC;
        '''

select_data(query).style.hide_index()

author,title
Пастернак Б.Л.,Лирика
Есенин С.А.,Черный человек
Достоевский Ф.М.,Братья Карамазовы
Достоевский Ф.М.,Игрок
Достоевский Ф.М.,Идиот
Булгаков М.А.,Белая гвардия
Булгаков М.А.,Мастер и Маргарита


## Групповые операции

In [27]:
query = '''
        SELECT DISTINCT amount
        FROM book;
        '''

select_data(query).style.hide_index()

amount
3
12
13
2
10
15
4


In [28]:
query = '''
        SELECT author AS 'Автор', COUNT(author) AS 'Различных_книг', SUM(amount) AS 'Количество_экземпляров'
        FROM book
        GROUP BY author;
        '''

select_data(query).style.hide_index()

Автор,Различных_книг,Количество_экземпляров
Булгаков М.А.,2,15.000000
Достоевский Ф.М.,3,25.000000
Есенин С.А.,2,27.000000
Пастернак Б.Л.,1,4.000000


In [29]:
query = '''
        SELECT author, MIN(price) AS 'Минимальная цена', MAX(price) AS 'Максимальная цена', AVG(price) AS 'Средняя цена'
        FROM book
        GROUP BY author;
        '''

select_data(query).style.hide_index()

author,Минимальная цена,Максимальная цена,Средняя цена
Булгаков М.А.,513.480000,670.990000,592.235000
Достоевский Ф.М.,387.400000,799.010000,539.620000
Есенин С.А.,541.690000,650.000000,595.845000
Пастернак Б.Л.,518.990000,518.990000,518.990000


In [30]:
query = '''
        SELECT author, round(SUM(price*amount), 2) AS 'Стоимость', round(SUM(price*amount) * 0.18 / (1 + 0.18), 2) AS 'НДС',
               round(SUM(price*amount) / (1 + 0.18), 2) AS 'Стоимость_без_НДС'
        FROM book
        GROUP BY author;
        '''

select_data(query).style.hide_index()

author,Стоимость,НДС,Стоимость_без_НДС
Булгаков М.А.,8174.730000,1246.990000,6927.740000
Достоевский Ф.М.,10958.720000,1671.670000,9287.050000
Есенин С.А.,16250.280000,2478.860000,13771.420000
Пастернак Б.Л.,2075.960000,316.670000,1759.290000


In [31]:
query = '''
        SELECT MIN(price) AS 'Минимальная_цена', MAX(price) AS 'Максимальная_цена', ROUND(AVG(price), 2) AS 'Средняя_цена'
        FROM book;
        '''

select_data(query).style.hide_index()

Минимальная_цена,Максимальная_цена,Средняя_цена
387.400000,799.010000,564.250000


In [32]:
query = '''
        SELECT ROUND(AVG(price), 2) AS 'Средняя_цена', ROUND(SUM(price*amount), 2) AS 'Стоимость'
        FROM book
        WHERE amount BETWEEN 5 and 14;
        '''

select_data(query).style.hide_index()

Средняя_цена,Стоимость
468.760000,22022.740000


In [33]:
query = '''
        SELECT author, SUM(price*amount) AS 'Стоимость'
        FROM book
        WHERE title <> 'Идиот' and title <> 'Белая гвардия'
        GROUP BY author
        HAVING Стоимость > 5000
        ORDER BY Стоимость DESC;
        '''

select_data(query).style.hide_index()

author,Стоимость
Есенин С.А.,16250.280000
Достоевский Ф.М.,5922.520000


## Вложенные запросы

In [34]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE price <= (SELECT AVG(price) FROM book)
        ORDER BY price DESC;
        '''

select_data(query).style.hide_index()

author,title,price
Есенин С.А.,Черный человек,541.690000
Пастернак Б.Л.,Лирика,518.990000
Булгаков М.А.,Белая гвардия,513.480000
Достоевский Ф.М.,Игрок,432.450000
Достоевский Ф.М.,Идиот,387.400000


In [35]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE price - (SELECT MIN(price) FROM book) <= 150
        ORDER BY price;
        '''

select_data(query).style.hide_index()

author,title,price
Достоевский Ф.М.,Идиот,387.400000
Достоевский Ф.М.,Игрок,432.450000
Булгаков М.А.,Белая гвардия,513.480000
Пастернак Б.Л.,Лирика,518.990000


In [36]:
query = '''
        SELECT author, title, amount
        FROM book
        WHERE amount IN (SELECT amount FROM book GROUP BY amount HAVING COUNT(amount) = 1);
        '''

select_data(query).style.hide_index()

author,title,amount
Булгаков М.А.,Мастер и Маргарита,3
Достоевский Ф.М.,Идиот,13
Достоевский Ф.М.,Братья Карамазовы,2
Достоевский Ф.М.,Игрок,10
Есенин С.А.,Стихотворения и поэмы,15
Пастернак Б.Л.,Лирика,4


In [37]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE author = ANY (SELECT author FROM book GROUP BY author HAVING AVG(price) > (SELECT AVG(price) FROM book));
        '''

select_data(query).style.hide_index()

author,title,price
Булгаков М.А.,Мастер и Маргарита,670.990000
Булгаков М.А.,Белая гвардия,513.480000
Есенин С.А.,Стихотворения и поэмы,650.000000
Есенин С.А.,Черный человек,541.690000


In [38]:
query = '''
        SELECT title, author, amount, (SELECT MAX(amount) FROM book) - amount AS 'Заказ'
        FROM book
        WHERE amount <> (SELECT MAX(amount) FROM book);
        '''

select_data(query).style.hide_index()

title,author,amount,Заказ
Мастер и Маргарита,Булгаков М.А.,3,12
Белая гвардия,Булгаков М.А.,12,3
Идиот,Достоевский Ф.М.,13,2
Братья Карамазовы,Достоевский Ф.М.,2,13
Игрок,Достоевский Ф.М.,10,5
Лирика,Пастернак Б.Л.,4,11
Черный человек,Есенин С.А.,12,3


In [39]:
query = '''
        SELECT title, author, amount, (SELECT MAX(amount) FROM book) - amount AS 'Заказ'
        FROM book
        WHERE amount <> (SELECT MAX(amount) FROM book);
        '''

select_data(query).style.hide_index()

title,author,amount,Заказ
Мастер и Маргарита,Булгаков М.А.,3,12
Белая гвардия,Булгаков М.А.,12,3
Идиот,Достоевский Ф.М.,13,2
Братья Карамазовы,Достоевский Ф.М.,2,13
Игрок,Достоевский Ф.М.,10,5
Лирика,Пастернак Б.Л.,4,11
Черный человек,Есенин С.А.,12,3


## Запросы корректировки данных

In [40]:
# query = '''
#         CREATE TABLE supply(supply_id INT PRIMARY KEY AUTO_INCREMENT,
#                             title VARCHAR(50),
#                             author VARCHAR(30),
#                             price DECIMAL(8, 2),
#                             amount INT);
#         '''
# create_table(query)

In [41]:
# query = '''
#         INSERT INTO supply (title, author, price, amount) VALUES ('Лирика', 'Пастернак Б.Л.', 518.99, 2),
#                                                                  ('Черный человек', 'Есенин С.А.', 570.20, 6),
#                                                                  ('Белая гвардия', 'Булгаков М.А.', 540.50, 7),
#                                                                  ('Идиот', 'Достоевский Ф.М.', 360.80, 3);
#         '''
# insert_values(query)

In [42]:
query = '''
        SELECT title, author, price, amount
        FROM supply
        '''

select_data(query).style.hide_index()

title,author,price,amount
Черный человек,Есенин С.А.,570.200000,6
Белая гвардия,Булгаков М.А.,540.500000,7
Идиот,Достоевский Ф.М.,360.800000,3


In [43]:
query = '''
        INSERT INTO book (title, author, price, amount)
            SELECT title, author, price, amount
            FROM supply
            WHERE author NOT IN ('Булгаков М.А.', 'Достоевский Ф.М.');
        '''

insert_values(query)

In [44]:
query = '''
        SELECT title, author, price, amount
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount
Мастер и Маргарита,Булгаков М.А.,670.990000,3
Белая гвардия,Булгаков М.А.,513.480000,12
Идиот,Достоевский Ф.М.,387.400000,13
Братья Карамазовы,Достоевский Ф.М.,799.010000,2
Игрок,Достоевский Ф.М.,432.450000,10
Стихотворения и поэмы,Есенин С.А.,650.000000,15
Лирика,Пастернак Б.Л.,518.990000,4
Черный человек,Есенин С.А.,541.690000,12
Черный человек,Есенин С.А.,570.200000,6


In [45]:
query = '''
        INSERT INTO book (title, author, price, amount)
               SELECT title, author, price, amount
               FROM supply
               WHERE author NOT IN (SELECT author FROM book);
        '''

insert_values(query)

In [46]:
query = '''
        SELECT title, author, price, amount
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount
Мастер и Маргарита,Булгаков М.А.,670.990000,3
Белая гвардия,Булгаков М.А.,513.480000,12
Идиот,Достоевский Ф.М.,387.400000,13
Братья Карамазовы,Достоевский Ф.М.,799.010000,2
Игрок,Достоевский Ф.М.,432.450000,10
Стихотворения и поэмы,Есенин С.А.,650.000000,15
Лирика,Пастернак Б.Л.,518.990000,4
Черный человек,Есенин С.А.,541.690000,12
Черный человек,Есенин С.А.,570.200000,6


In [47]:
query = '''
        UPDATE book SET price = 0.9 * price
                    WHERE amount BETWEEN 5 and 10;
        '''

insert_values(query)

In [48]:
query = '''
        SELECT title, author, price, amount
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount
Мастер и Маргарита,Булгаков М.А.,670.990000,3
Белая гвардия,Булгаков М.А.,513.480000,12
Идиот,Достоевский Ф.М.,387.400000,13
Братья Карамазовы,Достоевский Ф.М.,799.010000,2
Игрок,Достоевский Ф.М.,389.210000,10
Стихотворения и поэмы,Есенин С.А.,650.000000,15
Лирика,Пастернак Б.Л.,518.990000,4
Черный человек,Есенин С.А.,541.690000,12
Черный человек,Есенин С.А.,513.180000,6


In [49]:

query = '''
        ALTER TABLE book
        ADD COLUMN buy INT DEFAULT 0;
        '''

insert_values(query)

ProgrammingError(1060, "1060 (42S21): Duplicate column name 'buy'", '42S21')


In [50]:
query = '''
        UPDATE book SET buy = IF(title = 'Белая гвардия', 3, IF(title = 'Идиот', 8, IF(title = 'Стихотворения и поэмы', 15, buy)));
        '''

insert_values(query)

In [51]:
query = '''
        UPDATE book SET buy = IF(buy <= amount, buy, amount);
        '''

insert_values(query)

In [52]:
query = '''
        SELECT title, author, price, amount, buy
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount,buy
Мастер и Маргарита,Булгаков М.А.,670.990000,3,0
Белая гвардия,Булгаков М.А.,513.480000,12,3
Идиот,Достоевский Ф.М.,387.400000,13,8
Братья Карамазовы,Достоевский Ф.М.,799.010000,2,0
Игрок,Достоевский Ф.М.,389.210000,10,0
Стихотворения и поэмы,Есенин С.А.,650.000000,15,15
Лирика,Пастернак Б.Л.,518.990000,4,0
Черный человек,Есенин С.А.,541.690000,12,0
Черный человек,Есенин С.А.,513.180000,6,0


In [53]:
query = '''
        UPDATE book, supply SET book.amount = book.amount + supply.amount,
                                book.price = IF(book.price <> supply.price, round((book.price + supply.price) / 2, 2), book.price)
        WHERE book.title = supply.title and book.author = supply.author;
        '''

insert_values(query)

In [54]:
query = '''
        SELECT title, author, price, amount, buy
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount,buy
Мастер и Маргарита,Булгаков М.А.,670.990000,3,0
Белая гвардия,Булгаков М.А.,526.990000,19,3
Идиот,Достоевский Ф.М.,374.100000,16,8
Братья Карамазовы,Достоевский Ф.М.,799.010000,2,0
Игрок,Достоевский Ф.М.,389.210000,10,0
Стихотворения и поэмы,Есенин С.А.,650.000000,15,15
Лирика,Пастернак Б.Л.,518.990000,4,0
Черный человек,Есенин С.А.,555.950000,18,0
Черный человек,Есенин С.А.,541.690000,12,0


In [55]:
query = '''
        DELETE FROM supply
        WHERE title IN (SELECT title FROM book WHERE book.price = supply.price);
        '''

insert_values(query)

In [56]:
query = '''
        SELECT title, author, price, amount
        FROM supply
        '''

select_data(query).style.hide_index()

title,author,price,amount
Черный человек,Есенин С.А.,570.200000,6
Белая гвардия,Булгаков М.А.,540.500000,7
Идиот,Достоевский Ф.М.,360.800000,3


In [51]:
# query = '''
#         CREATE TABLE ordering AS
#                 SELECT author, title, 
#                        (SELECT round(AVG(amount)) FROM book) AS amount
#                 FROM book
#                 WHERE amount < (SELECT round(AVG(amount)) FROM book);
#         '''

# create_table(query)

In [57]:
query = '''
        SELECT *
        FROM ordering;
        '''

select_data(query).style.hide_index()

author,title,amount
Булгаков М.А.,Мастер и Маргарита,9.000000
Достоевский Ф.М.,Братья Карамазовы,9.000000
Пастернак Б.Л.,Лирика,9.000000


## Таблица "Командировки", запросы на выборку

In [53]:
# query = '''
#         CREATE TABLE trip (
#                              trip_id INT PRIMARY KEY AUTO_INCREMENT, 
#                              name VARCHAR(30), 
#                              city VARCHAR(25), 
#                              per_diem DECIMAL (8,2), 
#                              date_first DATE, 
#                              date_last DATE 
#                           );
#         '''

# create_table(query)

In [56]:
# query = '''
#         INSERT INTO trip (name, city, per_diem, date_first, date_last)
#         VALUES ('Абрамова К.А.', 'Владивосток', 450.00, STR_TO_DATE('2020-07-02', '%Y-%m-%d'), STR_TO_DATE('2020-07-13', '%Y-%m-%d')),
#                ('Федорова А.Ю.', 'Томск', 450.00, STR_TO_DATE('2020-06-20', '%Y-%m-%d'), STR_TO_DATE('2020-06-26', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Санкт-Петербург', 700.00, STR_TO_DATE('2020-05-28', '%Y-%m-%d'), STR_TO_DATE('2020-06-04', '%Y-%m-%d')),
#                ('Федорова А.Ю.', 'Новосибирск', 450.00, STR_TO_DATE('2020-05-25', '%Y-%m-%d'), STR_TO_DATE('2020-06-04', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Москва', 700.00, STR_TO_DATE('2020-04-06', '%Y-%m-%d'), STR_TO_DATE('2020-04-14', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Москва', 700.00, STR_TO_DATE('2020-02-23', '%Y-%m-%d'), STR_TO_DATE('2020-03-01', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Владивосток', 450.00, STR_TO_DATE('2020-01-14', '%Y-%m-%d'), STR_TO_DATE('2020-01-27', '%Y-%m-%d'));
#         '''

# insert_values(query)

In [58]:
# query = '''
#         INSERT INTO trip (name, city, per_diem, date_first, date_last)
#         VALUES ('Баранов П.Е.', 'Москва', 700.00, STR_TO_DATE('2020-01-12', '%Y-%m-%d'), STR_TO_DATE('2020-01-17', '%Y-%m-%d')),
#                ('Семенов И.В.', 'Москва', 700.00, STR_TO_DATE('2020-01-23', '%Y-%m-%d'), STR_TO_DATE('2020-01-31', '%Y-%m-%d')),
#                ('Ильиных Г.Р.', 'Владивосток', 450.00, STR_TO_DATE('2020-01-12', '%Y-%m-%d'), STR_TO_DATE('2020-02-02', '%Y-%m-%d')),
#                ('Колесов С.П.', 'Москва', 700.00, STR_TO_DATE('2020-02-01', '%Y-%m-%d'), STR_TO_DATE('2020-02-06', '%Y-%m-%d')),
#                ('Баранов П.Е.', 'Москва', 700.00, STR_TO_DATE('2020-02-14', '%Y-%m-%d'), STR_TO_DATE('2020-02-22', '%Y-%m-%d')),
#                ('Лебедев Т.К.', 'Москва', 700.00, STR_TO_DATE('2020-03-03', '%Y-%m-%d'), STR_TO_DATE('2020-03-06', '%Y-%m-%d')),
#                ('Колесов С.П.', 'Новосибирск', 450.00, STR_TO_DATE('2020-02-27', '%Y-%m-%d'), STR_TO_DATE('2020-03-12', '%Y-%m-%d')),
#                ('Семенов И.В.', 'Санкт-Петербург', 700.00, STR_TO_DATE('2020-03-29', '%Y-%m-%d'), STR_TO_DATE('2020-04-05', '%Y-%m-%d')),
#                ('Баранов П.Е.', 'Новосибирск', 450.00, STR_TO_DATE('2020-04-18', '%Y-%m-%d'), STR_TO_DATE('2020-05-04', '%Y-%m-%d')),
#                ('Лебедев Т.К.', 'Томск', 450.00, STR_TO_DATE('2020-05-20', '%Y-%m-%d'), STR_TO_DATE('2020-05-31', '%Y-%m-%d')),
#                ('Семенов И.В.', 'Санкт-Петербург', 700.00, STR_TO_DATE('2020-06-01', '%Y-%m-%d'), STR_TO_DATE('2020-06-03', '%Y-%m-%d')),
#                ('Колесов С.П.', 'Новосибирск', 450.00, STR_TO_DATE('2020-06-03', '%Y-%m-%d'), STR_TO_DATE('2020-06-12', '%Y-%m-%d')),
#                ('Баранов П.Е.', 'Воронеж', 450.00, STR_TO_DATE('2020-07-19', '%Y-%m-%d'), STR_TO_DATE('2020-07-25', '%Y-%m-%d'));
               
#         '''

# insert_values(query)

In [59]:
query = '''
        SELECT name, city, per_diem, date_first, date_last
        FROM trip
        WHERE name LIKE '%а %'
        ORDER BY date_last DESC;
        '''

select_data(query).style.hide_index()

name,city,per_diem,date_first,date_last
Абрамова К.А.,Владивосток,450.000000,2020-07-02,2020-07-13
Федорова А.Ю.,Томск,450.000000,2020-06-20,2020-06-26
Абрамова К.А.,Санкт-Петербург,700.000000,2020-05-28,2020-06-04
Федорова А.Ю.,Новосибирск,450.000000,2020-05-25,2020-06-04
Абрамова К.А.,Москва,700.000000,2020-04-06,2020-04-14
Абрамова К.А.,Москва,700.000000,2020-02-23,2020-03-01
Абрамова К.А.,Владивосток,450.000000,2020-01-14,2020-01-27


In [60]:
query = '''
        SELECT name
        FROM trip
        WHERE city = 'Москва'
        GROUP BY name
        ORDER BY name;
        '''

select_data(query).style.hide_index()

name
Абрамова К.А.
Баранов П.Е.
Колесов С.П.
Лебедев Т.К.
Семенов И.В.


In [61]:
query = '''
        SELECT city, COUNT(city) AS 'Количество'
        FROM trip
        GROUP BY city
        ORDER BY city;
        '''

select_data(query).style.hide_index()

city,Количество
Владивосток,3
Воронеж,1
Москва,7
Новосибирск,4
Санкт-Петербург,3
Томск,2


In [62]:
query = '''
        SELECT city, COUNT(city) AS 'Количество'
        FROM trip
        GROUP BY city
        ORDER BY COUNT(city) DESC
        LIMIT 2;
        '''

select_data(query).style.hide_index()

city,Количество
Москва,7
Новосибирск,4


In [63]:
query = '''
        SELECT name, city, DATEDIFF(date_last, date_first) + 1 AS 'Длительность'
        FROM trip
        WHERE city NOT IN ('Москва', 'Санкт-Петербург')
        ORDER BY DATEDIFF(date_last, date_first) + 1 DESC, city DESC;
        '''

select_data(query).style.hide_index()

name,city,Длительность
Ильиных Г.Р.,Владивосток,22
Баранов П.Е.,Новосибирск,17
Колесов С.П.,Новосибирск,15
Абрамова К.А.,Владивосток,14
Лебедев Т.К.,Томск,12
Абрамова К.А.,Владивосток,12
Федорова А.Ю.,Новосибирск,11
Колесов С.П.,Новосибирск,10
Федорова А.Ю.,Томск,7
Баранов П.Е.,Воронеж,7


In [64]:
query = '''
        SELECT name, city, date_first, date_last
        FROM trip
        WHERE DATEDIFF(date_last, date_first) + 1 = (SELECT MIN(DATEDIFF(date_last, date_first) + 1) FROM trip);
        '''

select_data(query).style.hide_index()

name,city,date_first,date_last
Семенов И.В.,Санкт-Петербург,2020-06-01,2020-06-03


In [65]:
query = '''
        SELECT name, city, date_first, date_last
        FROM trip
        WHERE MONTH(date_last) = MONTH(date_first)
        ORDER BY city, name;
        '''

select_data(query).style.hide_index()

name,city,date_first,date_last
Абрамова К.А.,Владивосток,2020-07-02,2020-07-13
Абрамова К.А.,Владивосток,2020-01-14,2020-01-27
Баранов П.Е.,Воронеж,2020-07-19,2020-07-25
Абрамова К.А.,Москва,2020-04-06,2020-04-14
Баранов П.Е.,Москва,2020-01-12,2020-01-17
Баранов П.Е.,Москва,2020-02-14,2020-02-22
Колесов С.П.,Москва,2020-02-01,2020-02-06
Лебедев Т.К.,Москва,2020-03-03,2020-03-06
Семенов И.В.,Москва,2020-01-23,2020-01-31
Колесов С.П.,Новосибирск,2020-06-03,2020-06-12


In [66]:
query = '''
        SELECT MONTHNAME(date_first) AS 'Месяц', COUNT(MONTH(date_first)) AS 'Количество'
        FROM trip
        GROUP BY Месяц
        ORDER BY Количество DESC, Месяц ASC;
        '''

select_data(query).style.hide_index()

Месяц,Количество
February,4
January,4
June,3
May,3
April,2
July,2
March,2


In [67]:
query = '''
        SELECT name, city, date_first, (DATEDIFF(date_last, date_first) + 1) * per_diem AS 'Сумма'
        FROM trip
        WHERE MONTH(date_first) IN (2, 3) AND YEAR(date_first) = 2020
        ORDER BY name, Сумма DESC;
        '''

select_data(query).style.hide_index()

name,city,date_first,Сумма
Абрамова К.А.,Москва,2020-02-23,5600.000000
Баранов П.Е.,Москва,2020-02-14,6300.000000
Колесов С.П.,Новосибирск,2020-02-27,6750.000000
Колесов С.П.,Москва,2020-02-01,4200.000000
Лебедев Т.К.,Москва,2020-03-03,2800.000000
Семенов И.В.,Санкт-Петербург,2020-03-29,5600.000000


In [68]:
query = '''
        SELECT name, SUM((DATEDIFF(date_last, date_first) + 1) * per_diem) AS 'Сумма'
        FROM trip
        WHERE name IN (SELECT name FROM trip GROUP BY name HAVING COUNT(name) > 3)
        GROUP BY name
        ORDER BY Сумма DESC;
        '''

select_data(query).style.hide_index()

name,Сумма
Абрамова К.А.,29200.000000
Баранов П.Е.,21300.000000


## Таблица "Нарушения ПДД", запросы корректировки

In [73]:
# query = '''
#         CREATE TABLE fine (
#                              fine_id INT PRIMARY KEY AUTO_INCREMENT, 
#                              name VARCHAR(30), 
#                              number_plate VARCHAR(6),
#                              violation VARCHAR(50),
#                              sum_fine DECIMAL (8,2), 
#                              date_violation DATE, 
#                              date_payment DATE 
#                           );
#         '''

# create_table(query)

In [74]:
query = '''
        INSERT INTO fine (name, number_plate, violation, sum_fine, date_violation, date_payment)
        VALUES ('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости (от 40 до 60)', 500.00, STR_TO_DATE('2020-01-12', '%Y-%m-%d'), STR_TO_DATE('2020-01-17', '%Y-%m-%d')),
               ('Абрамова К.А.', 'О111АВ', 'Проезд на запрещающий сигнал', 1000.00, STR_TO_DATE('2020-01-14', '%Y-%m-%d'), STR_TO_DATE('2020-02-27', '%Y-%m-%d')),
               ('Яковлев Г.Р.', 'Т330ТТ', 'Превышение скорости (от 20 до 40)', 500.00, STR_TO_DATE('2020-01-23', '%Y-%m-%d'), STR_TO_DATE('2020-02-23', '%Y-%m-%d')),
               ('Яковлев Г.Р.', 'М701АА', 'Превышение скорости (от 20 до 40)', NULL, STR_TO_DATE('2020-01-12', '%Y-%m-%d'), NULL),
               ('Колесов С.П.', 'К892АХ', 'Превышение скорости (от 20 до 40)', NULL, STR_TO_DATE('2020-02-01', '%Y-%m-%d'), NULL),
               ('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости (от 40 до 60)', NULL, STR_TO_DATE('2020-02-14', '%Y-%m-%d'), NULL),
               ('Абрамова К.А.', 'О111АВ', 'Проезд на запрещающий сигнал', NULL, STR_TO_DATE('2020-02-23', '%Y-%m-%d'), NULL),
               ('Яковлев Г.Р.', 'Т330ТТ', 'Проезд на запрещающий сигнал', NULL, STR_TO_DATE('2020-03-03', '%Y-%m-%d'), NULL);
        '''

insert_values(query)

In [76]:
# query = '''
#         CREATE TABLE traffic_violation (
#                              violation_id INT PRIMARY KEY AUTO_INCREMENT, 
#                              violation VARCHAR(50), 
#                              sum_fine DECIMAL (8,2)
#                           );
#         '''

# create_table(query)

In [77]:
# query = '''
#         INSERT INTO traffic_violation (violation, sum_fine)
#         VALUES ('Превышение скорости (от 20 до 40)', 500.00),
#                ('Превышение скорости (от 40 до 60)', 1000.00),
#                ('Проезд на запрещающий сигнал', 1000.00);
#         '''

# insert_values(query)

In [79]:
query = '''
        SELECT *
        FROM fine
        '''

select_data(query).style.hide_index()

fine_id,name,number_plate,violation,sum_fine,date_violation,date_payment
1,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),500.000000,2020-01-12,2020-01-17
2,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.000000,2020-01-14,2020-02-27
3,Яковлев Г.Р.,Т330ТТ,Превышение скорости (от 20 до 40),500.000000,2020-01-23,2020-02-23
4,Яковлев Г.Р.,М701АА,Превышение скорости (от 20 до 40),nan,2020-01-12,None
5,Колесов С.П.,К892АХ,Превышение скорости (от 20 до 40),nan,2020-02-01,None
6,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),nan,2020-02-14,None
7,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,nan,2020-02-23,None
8,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,nan,2020-03-03,None


In [78]:
query = '''
        SELECT *
        FROM traffic_violation
        '''

select_data(query).style.hide_index()

violation_id,violation,sum_fine
1,Превышение скорости (от 20 до 40),500.000000
2,Превышение скорости (от 40 до 60),1000.000000
3,Проезд на запрещающий сигнал,1000.000000


In [80]:
query = '''
        UPDATE fine f, traffic_violation tv SET f.sum_fine = tv.sum_fine
        WHERE f.sum_fine IS Null and f.violation = tv.violation;
        '''

insert_values(query)

In [81]:
query = '''
        SELECT *
        FROM fine
        '''

select_data(query).style.hide_index()

fine_id,name,number_plate,violation,sum_fine,date_violation,date_payment
1,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),500.000000,2020-01-12,2020-01-17
2,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.000000,2020-01-14,2020-02-27
3,Яковлев Г.Р.,Т330ТТ,Превышение скорости (от 20 до 40),500.000000,2020-01-23,2020-02-23
4,Яковлев Г.Р.,М701АА,Превышение скорости (от 20 до 40),500.000000,2020-01-12,None
5,Колесов С.П.,К892АХ,Превышение скорости (от 20 до 40),500.000000,2020-02-01,None
6,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),1000.000000,2020-02-14,None
7,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.000000,2020-02-23,None
8,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,1000.000000,2020-03-03,None


In [82]:
query = '''
        SELECT name, number_plate, violation
        FROM fine
        GROUP BY name, number_plate, violation
        HAVING COUNT(number_plate) > 1;
        '''

select_data(query).style.hide_index()

name,number_plate,violation
Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал
Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60)


In [83]:
query = '''
        UPDATE fine, (SELECT name, number_plate, violation FROM fine GROUP BY name, number_plate, violation HAVING COUNT(number_plate) > 1) query_in
        SET fine.sum_fine = 2 * fine.sum_fine
        WHERE fine.date_payment IS Null and fine.name = query_in.name AND fine.number_plate = query_in.number_plate AND fine.violation = query_in.violation;
        '''

insert_values(query)

In [84]:
query = '''
        SELECT *
        FROM fine
        '''

select_data(query).style.hide_index()

fine_id,name,number_plate,violation,sum_fine,date_violation,date_payment
1,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),500.000000,2020-01-12,2020-01-17
2,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.000000,2020-01-14,2020-02-27
3,Яковлев Г.Р.,Т330ТТ,Превышение скорости (от 20 до 40),500.000000,2020-01-23,2020-02-23
4,Яковлев Г.Р.,М701АА,Превышение скорости (от 20 до 40),500.000000,2020-01-12,None
5,Колесов С.П.,К892АХ,Превышение скорости (от 20 до 40),500.000000,2020-02-01,None
6,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),2000.000000,2020-02-14,None
7,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,2000.000000,2020-02-23,None
8,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,1000.000000,2020-03-03,None


In [86]:
# query = '''
#         CREATE TABLE payment (
#                              payment_id INT PRIMARY KEY AUTO_INCREMENT, 
#                              name VARCHAR(30), 
#                              number_plate VARCHAR(6),
#                              violation VARCHAR(50),
#                              date_violation DATE, 
#                              date_payment DATE 
#                           );
#         '''

# create_table(query)

In [87]:
query = '''
        INSERT INTO payment (name, number_plate, violation, date_violation, date_payment)
        VALUES ('Яковлев Г.Р.', 'М701АА', 'Превышение скорости (от 20 до 40)', STR_TO_DATE('2020-01-12', '%Y-%m-%d'), STR_TO_DATE('2020-01-22', '%Y-%m-%d')),
               ('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости (от 40 до 60)', STR_TO_DATE('2020-02-14', '%Y-%m-%d'), STR_TO_DATE('2020-03-15', '%Y-%m-%d')),
               ('Яковлев Г.Р.', 'Т330ТТ', 'Проезд на запрещающий сигнал', STR_TO_DATE('2020-03-03', '%Y-%m-%d'), STR_TO_DATE('2020-03-21', '%Y-%m-%d'));
        '''

insert_values(query)

In [88]:
query = '''
        UPDATE fine, payment SET fine.sum_fine = IF(DATEDIFF(payment.date_payment, payment.date_violation) + 1 < 20 , 0.5 * fine.sum_fine, fine.sum_fine), fine.date_payment = payment.date_payment
        WHERE fine.date_payment IS Null and fine.name = payment.name AND fine.number_plate = payment.number_plate AND fine.violation = payment.violation;
        '''

insert_values(query)

In [89]:
query = '''
        SELECT *
        FROM fine
        '''

select_data(query).style.hide_index()

fine_id,name,number_plate,violation,sum_fine,date_violation,date_payment
1,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),500.000000,2020-01-12,2020-01-17
2,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.000000,2020-01-14,2020-02-27
3,Яковлев Г.Р.,Т330ТТ,Превышение скорости (от 20 до 40),500.000000,2020-01-23,2020-02-23
4,Яковлев Г.Р.,М701АА,Превышение скорости (от 20 до 40),250.000000,2020-01-12,2020-01-22
5,Колесов С.П.,К892АХ,Превышение скорости (от 20 до 40),500.000000,2020-02-01,None
6,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),2000.000000,2020-02-14,2020-03-15
7,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,2000.000000,2020-02-23,None
8,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,500.000000,2020-03-03,2020-03-21


In [90]:
# query = '''
#         CREATE TABLE back_payment AS
#             SELECT name, number_plate, violation, sum_fine, date_violation
#             FROM fine
#             WHERE date_payment IS Null;
#         '''

# create_table(query)

In [91]:
query = '''
        SELECT *
        FROM back_payment;
        '''

select_data(query).style.hide_index()

name,number_plate,violation,sum_fine,date_violation
Колесов С.П.,К892АХ,Превышение скорости (от 20 до 40),500.000000,2020-02-01
Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,2000.000000,2020-02-23


In [92]:
query = '''
        DELETE FROM fine
        WHERE date_violation < '2020-02-01';
        '''

insert_values(query)

In [93]:
query = '''
        SELECT *
        FROM fine;
        '''

select_data(query).style.hide_index()

fine_id,name,number_plate,violation,sum_fine,date_violation,date_payment
5,Колесов С.П.,К892АХ,Превышение скорости (от 20 до 40),500.000000,2020-02-01,None
6,Баранов П.Е.,Р523ВТ,Превышение скорости (от 40 до 60),2000.000000,2020-02-14,2020-03-15
7,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,2000.000000,2020-02-23,None
8,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,500.000000,2020-03-03,2020-03-21
